In [ ]:
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix
# !jupyter nbextension enable --py tqdm.notebook
# !jupyter nbextension enable --py tqdm.notebook --sys-prefix

In [1]:
import pandas as pd
import numpy as np
import os
from transformers import pipeline
from nltk.tokenize import sent_tokenize
import nltk
from tqdm.notebook import tqdm

# nltk.download('
#punkt')

In [2]:
os.chdir("..")
DATA_PATH = "ArticleData.csv"
MODEL_NAME = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

In [3]:
model_pipeline = pipeline("sentiment-analysis", model=MODEL_NAME, return_all_scores=True)

c:\LSE\DS\Twitter-Acquisition-Sentiment-Analysis\.venv\lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [4]:
df = pd.read_csv(DATA_PATH)
df = df.loc[~pd.isnull(df["DATE"])]

youtube_df = df.loc[df["OUTLET"] == "YouTube"]
news_df = df.loc[df["OUTLET"] != "YouTube"]

C:\Users\HP ENVY 13.3\AppData\Local\Temp\ipykernel_19292\3112371530.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_PATH)


In [6]:
news_df

,DATE,OUTLET,AUTHOR,DESCRIPTION,TEXT
184551,11-11-2023,bing,Yahoo News UK,Tesla has spent the past year slashing its pri...,Elon Musk has started an electric-vehicle pric...
184552,12-11-2023,bing,Daily Mail,Elon Musk discouraged employees from wearing y...,By James Gordon For Dailymail.com and Reuters...
184553,22-10-2023,bing,Daily Mail,New research indicates the obesity drug Wegovy...,By Noa Halff For Dailymail.Com and Associated ...
184554,01-11-2023,bing,Yahoo,"""PayPal mafia"" members Elon Musk, Peter Thiel,...",A group of early 2000s PayPal employees and fo...
184555,24-10-2023,bing,Yahoo,Tesla founder Elon Musk has a biopic on his li...,Tesla founder Elon Musk has a biopic on his li...
...,...,...,...,...,...
205461,10-11-2023,washington post,By Christian Davenport,NaN,"\nNow, more than six months later — after the ..."
205462,13-11-2023,washington post,Elizabeth Dwoskin Maeve Reston Hannah Knowles,NaN,"Copyright 2023, The Washington Post Co. All Ri..."
205463,13-11-2023,washington post,By Lili Loofbourow,NaN,"\n""A Murder at the End of the World"" courts th..."
205464,13-11-2023,washington post,By Laura ReileyKadir van Lohuizen/NOOR,NaN,\nThese exhortations worked: Total farm output...


In [5]:
def matching_sentences(sentences: list, phrases: list) -> list:
    """
    Function returns sentences that contain at least one phrase in the list of phrases.
    """
    matching_sentences = []

    for sentence in sentences:
        for phrase in phrases:
            if phrase in sentence:
                matching_sentences.append(sentence)
                break

    return matching_sentences

In [6]:
def parse_sentiment_scores(result):
    """
    Converts sentiment labels to score between -1 and 1 
    """
    labels = result[0]
    scores = [i["score"] for i in labels]
    idx = np.argmax(scores)
    if idx == 0:
        return labels[0]["score"] * -1
    elif idx == 1:
        return labels[2]["score"] - labels[0]["score"]
    elif idx ==2:
        return labels[2]["score"]

In [9]:
phrases = [
    "Elon",
    "Musk",
    "Twitter",
    "X",
]

sentiments = []
with open("sentiments.txt", 'a') as file:
    for text in tqdm(news_df["TEXT"]):
        sentence_sentiments = []
        try:
            sentences = sent_tokenize(text)
        except:
            sentiments.append(np.NaN)
            file.write(f"{np.NaN}\n")
            continue

        sentences = matching_sentences(sentences, phrases)

        if len(sentences) == 0:
            sentiments.append(np.NaN)
            continue
        for sentence in sentences:
            try:
                result = model_pipeline(sentence)
                sentiment = parse_sentiment_scores(result)
                sentence_sentiments.append(sentiment)
            except:
                continue
        
        try:
            s = np.mean(sentence_sentiments)
            sentiments.append(s)
            file.write(f"{s}\n")
        except:
            sentiments.append(np.NaN)
            file.write(f"{np.NaN}\n")
    
sentiments


  0%|          | 0/20915 [00:00<?, ?it/s]

c:\LSE\DS\Twitter-Acquisition-Sentiment-Analysis\.venv\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\LSE\DS\Twitter-Acquisition-Sentiment-Analysis\.venv\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[-0.1086931503030731,
 -0.1514810464542543,
 0.7430388995271642,
 0.0004136913272044954,
 0.04731788561184658,
 0.0005454076745081693,
 -3.847759944619611e-05,
 0.09993363719549961,
 -0.08301628725952728,
 -1.2284388503758237e-05,
 0.2038106198257689,
 0.16766247232226306,
 -0.035002783217542424,
 -0.02808378468450558,
 -0.17837685711943085,
 -0.4313502652803436,
 0.11159675690893588,
 -0.136300268508681,
 -0.24273495098064554,
 -0.000620626897944021,
 -0.018202857740872883,
 0.03086606543415242,
 0.1998183171997559,
 -0.03305318077888972,
 -0.04191647115132784,
 0.09080349973252548,
 0.13511232028196735,
 0.11822889940367531,
 0.012240615725204407,
 -0.36750778281784735,
 -0.03127878736228701,
 -8.022546483386707e-05,
 0.15861613184248805,
 -0.15006558386812685,
 -0.0018977431297268292,
 -0.3287089781333634,
 -2.975580719066784e-06,
 -0.0624611276343785,
 0.2811422159837093,
 -0.14109757041608514,
 0.0017111485707573594,
 -2.8762482543243095e-05,
 0.0004191235675534699,
 0.09439423742

In [11]:
news_df["SENTIMENT"] = sentiments

C:\Users\HP ENVY 13.3\AppData\Local\Temp\ipykernel_19292\3458534441.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df["SENTIMENT"] = sentiments


In [37]:
news_df.to_csv("NewsSentiment.csv", encoding="utf-8", index=False)

In [38]:
x = pd.read_csv("NewsSentiment.csv")
x

,DATE,OUTLET,AUTHOR,DESCRIPTION,TEXT,SENTIMENT
0,11-11-2023,bing,Yahoo News UK,Tesla has spent the past year slashing its pri...,Elon Musk has started an electric-vehicle pric...,-0.108693
1,12-11-2023,bing,Daily Mail,Elon Musk discouraged employees from wearing y...,By James Gordon For Dailymail.com and Reuters...,-0.151481
2,22-10-2023,bing,Daily Mail,New research indicates the obesity drug Wegovy...,By Noa Halff For Dailymail.Com and Associated ...,0.743039
3,01-11-2023,bing,Yahoo,"""PayPal mafia"" members Elon Musk, Peter Thiel,...",A group of early 2000s PayPal employees and fo...,0.000414
4,24-10-2023,bing,Yahoo,Tesla founder Elon Musk has a biopic on his li...,Tesla founder Elon Musk has a biopic on his li...,0.047318
...,...,...,...,...,...,...
20910,10-11-2023,washington post,By Christian Davenport,NaN,"\nNow, more than six months later — after the ...",0.152060
20911,13-11-2023,washington post,Elizabeth Dwoskin Maeve Reston Hannah Knowles,NaN,"Copyright 2023, The Washington Post Co. All Ri...",0.000033
20912,13-11-2023,washington post,By Lili Loofbourow,NaN,"\n""A Murder at the End of the World"" courts th...",-0.000040
20913,13-11-2023,washington post,By Laura ReileyKadir van Lohuizen/NOOR,NaN,\nThese exhortations worked: Total farm output...,-0.015120
